In [1]:
# https://www.kaggle.com/datasets/kristianperriu/genereclassification
# https://www.kaggle.com/datasets/karnikakapoor/lyrics

In [2]:
from sklearn.datasets import fetch_20newsgroups

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess

import spacy
from pprint import pprint
import pandas as pd

# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from','subject','re','edu','use'])

C:\Users\Daniel Atzberger\anaconda3\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "
C:\Users\Daniel Atzberger\anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [3]:
df = pd.read_csv("data/Lyrics/lyrics_dataframe_clean_1.csv")

In [4]:
df.head()

,Unnamed: 0,lyrics,genere
0,0,\nyo i never fucked wayne i never fucked drake...,rap
1,1,\nyo b they ready\nlet's go\n\nfeelin' myself ...,rap
2,2,\nuh mmm kyuh\nrip to big \nclassic shit\n\ni'...,rap
3,3,\nyou know yeah\ntouchin' yeah\nnight of\nyou ...,rap
4,4,\nayo look like i'm goin' for a swim\ndunked o...,rap


In [5]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

In [6]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

In [7]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc])
    return texts_out

In [8]:
data = df["lyrics"].tolist()
data_words = list(sent_to_words(data))

# Remove Stop Words
print("Start removing stop words")
data_words_nostops = remove_stopwords(data_words)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# conda install -c conda-forge spacy-model-en_core_web_sm
print("Installing spacy")
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
print("Start lemmatizing words")
#data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
data_lemmatized = lemmatization(data_words_nostops, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

Start removing stop words
Installing spacy
Start lemmatizing words


In [9]:
data_lemmatized_min_length = []

for sublist in data_lemmatized:
    # Use a list comprehension to filter out strings with less than two characters
    sublist = [word for word in sublist if len(word) > 2]
    data_lemmatized_min_length.append(sublist)

In [10]:
print(data_lemmatized_min_length[348])

['tay', 'keith', 'fuck', 'niggas', 'type', 'guy', 'like', 'since', 'middle', 'school', 'never', 'think', 'get', 'famous', 'always', 'know', 'cool', 'could', 'fuck', 'shawty', 'wanna', 'rude', 'eliantte', 'diamond', 'want', 'fine', 'jewel', 'hitter', 'dem', 'slidin', 'put', 'camera', 'crew', 'doin', 'bad', 'somebody', 'google', 'never', 'worry', 'bout', 'say', 'gon', 'put', 'overtime', 'doin', 'number', 'like', 'two', 'back', 'time', 'change', 'new', 'money', 'conversation', 'table', 'speak', 'fluently', 'stay', 'away', 'bad', 'vibe', 'see', 'nigga', 'still', 'give', 'couple', 'thousand', 'even', 'though', 'would', 'free', 'treat', 'like', 'rapper', 'cause', 'vibe', 'like', 'street', 'one', 'top', 'five', 'many', 'guy', 'real', 'barely', 'get', 'sleep', 'tryna', 'make', 'sure', 'everybody', 'eat', 'take', 'trip', 'overseas', 'buyin', 'drip', 'currency', 'fire', 'right', 'somethin', 'currently', 'bitch', 'tell', 'lie', 'tell', 'curvin', 'big', 'dawgs', 'gon', 'let', 'people', 'work', 'ge

In [11]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized_min_length)

# Create Corpus
texts = data_lemmatized_min_length

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View 
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 3), (4, 1), (5, 1), (6, 1), (7, 3), (8, 1), (9, 1), (10, 8), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 3), (19, 18), (20, 1), (21, 2), (22, 3), (23, 3), (24, 4), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 3), (32, 1), (33, 3), (34, 1), (35, 1), (36, 1), (37, 1), (38, 2), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 3), (45, 1), (46, 1), (47, 2), (48, 2), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 3), (58, 1), (59, 1), (60, 1), (61, 1), (62, 3), (63, 1), (64, 1), (65, 2), (66, 1), (67, 1), (68, 1), (69, 3), (70, 1), (71, 19), (72, 1), (73, 1), (74, 8), (75, 2), (76, 1), (77, 2), (78, 1), (79, 1), (80, 1), (81, 1), (82, 2), (83, 1), (84, 1), (85, 1), (86, 1), (87, 2), (88, 2), (89, 1), (90, 1), (91, 1), (92, 1), (93, 4), (94, 1), (95, 1), (96, 3), (97, 1), (98, 1), (99, 1), (100, 3), (101, 1), (102, 2), (103, 5), (104, 3), (105, 1), (106, 1), (107, 1), (108, 2), (109, 2), (110, 

In [12]:
# Number of documents
print(len(corpus))

# Size of the vocabulary
print(len(id2word))

10995
32758


In [13]:
import statistics

def sum_of_second_components(tuple_list):
    total_sum = 0
    for tup in tuple_list:
        total_sum += tup[1]  # Accessing the second component of each tuple
    return total_sum

lengths = []
for doc in corpus:
    lengths.append(sum_of_second_components(doc))
print(statistics.median(lengths))

135


### Topic Models

#### Vector Space Model (VSM)

In [ ]:
from scipy.sparse import dok_matrix

# Define function to convert Gensim corpus to a sparse pandas DataFrame
def corpus_to_sparse_dataframe(corpus):
    word_freq = dok_matrix((len(corpus), len(id2word)), dtype=int)

    for i, doc in enumerate(corpus):
        for word_id, freq in doc:
            word_freq[i, word_id] = freq

    dataframe = pd.DataFrame.sparse.from_spmatrix(word_freq)
    dataframe.columns = [id2word[word_id] for word_id in range(len(id2word))]
    return dataframe

In [ ]:
VSM = corpus_to_sparse_dataframe(corpus)

#### Vector Space Model & tf-idf (VSM & tf-idf)

In [ ]:
from gensim.models import TfidfModel

model = TfidfModel(corpus)  # fit model
tfidf_corpus = model[corpus]

#### Latent Semantic Indexing (LSI)

In [ ]:
from gensim.models import LsiModel

K = 12
lsi_model = LsiModel(corpus, id2word=id2word, num_topics=K)
lsi_model.print_topics(num_topics=K, num_words=10)

#### Latent Semantic Indexing & tf-idf (LSI & tf-idf)

In [ ]:
from gensim.models import LsiModel

K = 12
tfidf_lsi_model = LsiModel(tfidf_corpus, id2word=id2word, num_topics=K)
tfidf_lsi_model.print_topics(num_topics=K, num_words=10)

#### Non-Negative Matrix Factorization (NMF)

In [ ]:
# https://github.com/piskvorky/gensim/blob/develop/docs/notebooks/nmf_tutorial.ipynb
from gensim.models.nmf import Nmf

K = 12
nmf_model = Nmf(corpus, id2word=id2word, num_topics=K)
nmf_model.show_topics(num_topics=K, num_words=10)

#### Non-Negative Matrix Factorization & tf-idf (NMF & tf-idf)

In [ ]:
# https://github.com/piskvorky/gensim/blob/develop/docs/notebooks/nmf_tutorial.ipynb
from gensim.models.nmf import Nmf

K = 12
tfidf_nmf_model = Nmf(tfidf_corpus, id2word=id2word, num_topics=K)
tfidf_nmf_model.show_topics(num_topics=K, num_words=10)

#### Latent Dirichlet Allocation (LDA)

In [ ]:
# Build LDA model
K = 12
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=K,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=400,
                                           passes=30,
                                           alpha='auto',
                                           per_word_topics=True)

# Print the Keywords in the 3 topics
pprint(lda_model.print_topics())

### Miscallenous

In [ ]:
# Build LDA model
num_topics = 12
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=num_topics,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=400,
                                           passes=30,
                                           alpha='auto',
                                           per_word_topics=True)

# Print the Keywords in the 3 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

In [ ]:
rows = []
for doc in corpus:
    doc_top = []
    for t in lda_model.get_document_topics(doc, minimum_probability = 0):
        doc_top.append(t[1])
    rows.append(doc_top)

In [ ]:
document_topic_matrix = pd.DataFrame(rows)
#document_topic_matrix_sourcecode["identifier"] = df_sourcecode.iloc[:,0].tolist()
document_topic_matrix

In [ ]:
DTM = document_topic_matrix
Y = df["genere"].tolist()

In [ ]:
def string_to_int_mapping(strings):
    # Create a dictionary to store string to integer mapping
    string_to_int = {}
    int_sequence = []

    # Assign a unique integer to each unique string
    for string in strings:
        if string not in string_to_int:
            string_to_int[string] = len(string_to_int)  # Assign the next available integer
        # Append the corresponding integer to the sequence
        int_sequence.append(string_to_int[string])

    return int_sequence, string_to_int

# Example usage:
strings = ["apple", "banana", "apple", "orange", "banana"]
int_sequence, string_to_int = string_to_int_mapping(strings)
print("String to integer sequence:", int_sequence)
print("String to integer mapping:", string_to_int)

In [ ]:
Y_converted,_ = string_to_int_mapping(Y)

In [ ]:
# Dimensionality reduction 1: tSNE

import time
from sklearn.manifold import TSNE

#n_sne = 7000

time_start = time.time()
#tsne = TSNE(n_iter=300)
#tsne = TSNE(n_components=2, n_iter = 2000, perplexity=40)
tsne = TSNE(n_components=2)
tsne_results = tsne.fit_transform(DTM)

print ('t-SNE done! Time elapsed: {} seconds'.format(time.time()-time_start))

import matplotlib.pyplot as plt
%matplotlib inline
# Create the figure
fig = plt.figure( figsize=(8,8) )
ax = fig.add_subplot(1, 1, 1, title='TSNE' )
# Create the scatter
ax.scatter(
    x=tsne_results[:,0], 
    y=tsne_results[:,1], 
    c=Y_converted, 
    cmap=plt.cm.get_cmap('Paired'), 
    alpha=1.0)
plt.show()